In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import requests
import pprint as pp

# Import API key
from config import g_key

In [80]:
# File to Load
NYC1_data_to_load = "resources/NYC_Listings1.csv"
NYC2_data_to_load = "resources/NYC_Listings2.csv"
NYC3_data_to_load = "resources/NYC_Listings3.csv"

# Read School and Student Data File and store into Pandas Data Frames
NYC1_data = pd.read_csv(NYC1_data_to_load)
NYC2_data = pd.read_csv(NYC2_data_to_load)
NYC3_data = pd.read_csv(NYC3_data_to_load)

In [81]:
# Create a full NYC DF
NYC_data = pd.concat([NYC1_data, NYC2_data, NYC3_data], ignore_index=True)
NYC_data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,2.019080e+13,8/7/19,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2.0,1.0,0.0,1.0,0.39
1,3647,https://www.airbnb.com/rooms/3647,2.019080e+13,8/6/19,THE VILLAGE OF HARLEM....NEW YORK !,NaN,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,0.0,1.0,0.0,NaN
2,3831,https://www.airbnb.com/rooms/3831,2.019080e+13,8/6/19,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1.0,1.0,0.0,0.0,4.64
3,5022,https://www.airbnb.com/rooms/5022,2.019080e+13,8/6/19,Entire Apt: Spacious Studio/Loft by central park,NaN,Loft apartment with high ceiling and wood floo...,Loft apartment with high ceiling and wood floo...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.10
4,5099,https://www.airbnb.com/rooms/5099,2.019080e+13,8/6/19,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment is true New York ...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment is true New York ...,none,My neighborhood in Midtown East is called Murr...,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.60


In [82]:
# Review columns
column_names = list(NYC_data.columns)
column_names[:5]

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name']

In [109]:
# Select Columns for clean DataFrame
NYC_data['id']
NYC_data['name']
NYC_data['host_location']
NYC_data['latitude']
NYC_data['longitude']
NYC_data['property_type']
NYC_data['room_type']
NYC_data['accommodates']
NYC_data['amenities']
NYC_data['price']
NYC_data['availability_365']
NYC_data['number_of_reviews']
NYC_data['number_of_reviews_ltm']
NYC_data['review_scores_rating']
NYC_data['review_scores_location']
NYC_data['reviews_per_month']

Clean_NYC = NYC_data[['id','name','neighbourhood',
                      'latitude','longitude','property_type',
                      'room_type','accommodates','amenities',
                      'price','availability_365','number_of_reviews',
                     'number_of_reviews_ltm','review_scores_rating',
                      'review_scores_location','reviews_per_month']]
# Drop null values
Clean_NYC = Clean_NYC.dropna()

# Change price value to float
Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.lstrip('$'))
#Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.lstrip(','))
Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.replace(',',''))
Clean_NYC['price'] = Clean_NYC['price'].astype('float')

# Preview DataFrame
Clean_NYC.head()


,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month
0,2595,Skylit Midtown Castle,Midtown,40.75362,-73.98377,Apartment,Entire home/apt,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",225.0,288.0,46.0,12.0,95.0,10.0,0.39
2,3831,Cozy Entire Floor of Brownstone,Brooklyn,40.68514,-73.95976,Guest suite,Entire home/apt,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",89.0,212.0,274.0,70.0,90.0,10.0,4.64
3,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,40.79851,-73.94399,Apartment,Entire home/apt,1.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Elev...",80.0,0.0,9.0,2.0,93.0,9.0,0.10
4,5099,Large Cozy 1 BR Apartment In Midtown East,Midtown East,40.74767,-73.97500,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Buzzer/w...",200.0,127.0,75.0,10.0,89.0,9.0,0.60
5,5121,BlissArtsSpace!,Brooklyn,40.68688,-73.95596,Apartment,Private room,2.0,"{Wifi,""Air conditioning"",Kitchen,""Pets live on...",60.0,0.0,49.0,0.0,90.0,9.0,0.39


In [111]:
# GroupBY Neighborhood
neighborhoods = Clean_NYC.groupby('neighbourhood').describe()
neighborhoods.head()

accommodates                                                  \
                     count      mean       std  min   25%  50%   75%   max   
neighbourhood                                                                
Allerton              10.0  2.900000  0.994429  2.0  2.00  2.5  4.00   4.0   
Alphabet City        193.0  2.616580  1.495874  1.0  2.00  2.0  3.00  10.0   
Annadale               2.0  3.500000  0.707107  3.0  3.25  3.5  3.75   4.0   
Arrochar               1.0  4.000000       NaN  4.0  4.00  4.0  4.00   4.0   
Astoria              470.0  2.508511  1.608523  1.0  2.00  2.0  3.00  12.0   

              availability_365              ...  review_scores_rating         \
                         count        mean  ...                   75%    max   
neighbourhood                               ...                                
Allerton                  10.0  193.600000  ...                  98.5  100.0   
Alphabet City            193.0   71.922280  ...                  98.0  100.0   
Annadale                   2.0  204.000000  ...                  96.5   97.0   
Arrochar                   1.0  355.000000  ...                  80.0   80.0   
Astoria                  470.0  113.557447  ...                 100.0  100.0   

              reviews_per_month                                           \
                          count      mean       std   min     25%    50%   
neighbourhood                                                              
Allerton                   10.0  1.753000  1.501866  0.14  0.2275  2.190   
Alphabet City             193.0  1.142591  1.464204  0.02  0.1300  0.440   
Annadale                    2.0  3.635000  1.732412  2.41  3.0225  3.635   
Arrochar                    1.0  0.460000       NaN  0.46  0.4600  0.460   
Astoria                   470.0  1.119043  1.401495  0.02  0.1600  0.500   

                             
                  75%   max  
neighbourhood                
Allerton       2.3600  4.53  
Alphabet City  1.6700  6.38  
Annadale       4.2475  4.86  
Arrochar       0.4600  0.46  
Astoria        1.7075  7.75  

[5 rows x 88 columns]

In [112]:
neighborhoods_list = list(neighborhoods.index)
neighborhoods_list[:10]

['Allerton',
 'Alphabet City',
 'Annadale',
 'Arrochar',
 'Astoria',
 'Bath Beach',
 'Battery Park City',
 'Bay Ridge',
 'Baychester',
 'Bayside']

In [113]:
# Create a Dictionary with neighbourhood DataFrames
d = {}
for city in neighborhoods_list:
    checker = (Clean_NYC['neighbourhood'] == city)
    city_df = Clean_NYC.where(checker)
    city_df = city_df.dropna()
    d[city] = city_df


In [115]:
woodside_df = d['Woodside']
woodside_df.head()


,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month
77,18198.0,Little King of Queens,Woodside,40.75038,-73.90334,Apartment,Private room,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",70.0,237.0,25.0,1.0,100.0,9.0,0.22
1528,704838.0,An Oasis in the Big Apple 1,Woodside,40.74723,-73.89706,Apartment,Entire home/apt,6.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",120.0,289.0,62.0,24.0,95.0,10.0,1.60
1546,714049.0,An Oasis in the Big Apple 3,Woodside,40.74687,-73.89892,Apartment,Entire home/apt,7.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",120.0,289.0,55.0,20.0,90.0,9.0,0.66
2502,1325432.0,"Private Room in Woodside, Queens: Good Deal!",Woodside,40.74265,-73.89781,Apartment,Private room,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",79.0,335.0,63.0,11.0,96.0,10.0,0.95
4159,2803944.0,Lovely Private Bedroom in 2BR Apt,Woodside,40.74235,-73.90375,Apartment,Private room,2.0,"{Internet,Wifi,""Air conditioning"",""Wheelchair ...",55.0,0.0,1.0,0.0,80.0,10.0,0.02


In [121]:
# Create a Gmaps figure
fig = gmaps.figure()

locations = woodside_df[['latitude','longitude']]
weights = woodside_df['review_scores_location']
# Create the Heat Layer Object 
heat_layer = gmaps.heatmap_layer(locations=locations, weights=weights, dissipating=True)
heat_layer.point_radius = 7
# Add Heat Layer to Figure
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))